In [1]:
%config IPCompleter.greedy=True
import findspark
findspark.init()
from pyspark import SparkConf
from pyspark import SparkContext
import time

In [2]:
conf=SparkConf().setMaster("local[*]").setAppName("ElectronicProductsRecommendationSystem")
sc = SparkContext.getOrCreate(conf=conf) 

In [4]:
dosya_yolu = "file:///C:/Users/HP/Documents/mergedfile.txt"
rdd = sc.textFile(dosya_yolu)
rdd.count()

96000001

In [7]:
start_time = time.time()

import re

# Function to normalize a single line of text
def normalize_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuations and digits using regex
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove leading and trailing whitespaces
    lines =text.split('\n')
    lines = [line.strip() for line in lines if line.strip()]
    return lines

# Apply normalization to each line of the RDD
normalized_rdd = rdd.flatMap(normalize_text)

line_count= normalized_rdd.count()

# Count the number of lines after normalization
print("Number of lines after normalization:", line_count)



# Split each line into words and count the occurrences of each word
words_count_rdd = normalized_rdd.flatMap(lambda line: line.split()) \
                                 .map(lambda word: (word, 1))

word_counts=words_count_rdd.reduceByKey(lambda x, y: x + y)

# Sort the result by value in descending order
sorted_word_counts = word_counts.sortBy(lambda x: x[1], ascending=False)

top_10_words =sorted_word_counts.take(10)
print("Top 10 words:")
for word, count in top_10_words:
    print(f"{word}: {count}")
# Save the first 10 key-value pairs as a text file
output_directory =  r"file:///C:/Users/HP/Documents/output_12GB"
top_10_rdd=sc.parallelize(top_10_words)
top_10_rdd.saveAsTextFile(output_directory)


end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time

# Print the execution time
print("Execution time:", execution_time, "seconds")  

Number of lines after normalization: 96000000
Top 10 words:
the: 131545248
of: 57200160
to: 54364608
a: 48745728
in: 45462432
and: 43053024
said: 20726112
for: 20616864
that: 19393632
is: 16872480
Execution time: 1998.3753299713135 seconds
